In [28]:
from pypdf import PdfReader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_groq import ChatGroq
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser
from langchain.chains import RetrievalQA
from langchain_core.runnables import RunnablePassthrough, RunnableLambda

In [2]:
def get_pdf_text(pdf):
    pieces = []
    reader = PdfReader(pdf)
    for i, page in enumerate(reader.pages):
        text = page.extract_text()
        pieces.append(text or '')
        print(f'Página {i+1}:')
        print(text)
    return "".join(pieces)

In [3]:
path = r'C:\Users\Orçamento\Desktop\ENGENHORCA\CAIXA TERMINAL F650.pdf'
raw_text = get_pdf_text(path)
raw_text

Página 1:
Caixas Terminais
F6/1/P/6
Para filtros de alta eficiência, montagem no 
teto e montagem em parede
®
TROX DO BRASIL LTDA.
Rua Alvarenga, 2025  
05509-005 – São Paulo – SP
+55 (11) 3037-3900
trox-br@troxgroup.com
www.troxbrasil.com.br
 +55 11 97395 1627

Página 2:
Índice - Conteúdo - Tipos de produtos
Caixa Terminal F631
Caixa Terminal F640 
(na figura com difusor tipo VDW 676X54)
Caixa Terminal F660 
(na figura com difusor tipo FD)
Caixa Terminal F650
(na figura com difusor tipo ADLQ-A)
Caixa Terminal F670 
(na figura com grelha tipo AT-A)
Índice - Conteúdo -Tipos de produtos ........................... 2
Caixa Terminal F631 com Filtros de Alta Eficiência para 
montagem no teto .......................................................... 3
Caixa Terminal F650 com Filtros de Alta Eficiência para 
montagem no teto .......................................................... 4
Caixa Terminal F640 com Filtros de Alta Eficiência para 
montagem no teto .................................

'Caixas Terminais\nF6/1/P/6\nPara filtros de alta eficiência, montagem no \nteto e montagem em parede\n®\nTROX DO BRASIL LTDA.\nRua Alvarenga, 2025  \n05509-005 – São Paulo – SP\n+55 (11) 3037-3900\ntrox-br@troxgroup.com\nwww.troxbrasil.com.br\n\uf232 +55 11 97395 1627\nÍndice - Conteúdo - Tipos de produtos\nCaixa Terminal F631\nCaixa Terminal F640 \n(na figura com difusor tipo VDW 676X54)\nCaixa Terminal F660 \n(na figura com difusor tipo FD)\nCaixa Terminal F650\n(na figura com difusor tipo ADLQ-A)\nCaixa Terminal F670 \n(na figura com grelha tipo AT-A)\nÍndice - Conteúdo -Tipos de produtos ........................... 2\nCaixa Terminal F631 com Filtros de Alta Eficiência para \nmontagem no teto .......................................................... 3\nCaixa Terminal F650 com Filtros de Alta Eficiência para \nmontagem no teto .......................................................... 4\nCaixa Terminal F640 com Filtros de Alta Eficiência para \nmontagem no teto ....................

In [4]:
def get_text_chunks(raw_text):
    text_splitter = RecursiveCharacterTextSplitter(separators=["\n\n", "\n", " ", ""],chunk_size=300,chunk_overlap=50)
    texts = text_splitter.split_text(raw_text or '')
    return texts

In [5]:
text_chunks = get_text_chunks(raw_text)
text_chunks

['Caixas Terminais\nF6/1/P/6\nPara filtros de alta eficiência, montagem no \nteto e montagem em parede\n®\nTROX DO BRASIL LTDA.\nRua Alvarenga, 2025  \n05509-005 – São Paulo – SP\n+55 (11) 3037-3900\ntrox-br@troxgroup.com\nwww.troxbrasil.com.br\n\uf232 +55 11 97395 1627\nÍndice - Conteúdo - Tipos de produtos',
 'Índice - Conteúdo - Tipos de produtos\nCaixa Terminal F631\nCaixa Terminal F640 \n(na figura com difusor tipo VDW 676X54)\nCaixa Terminal F660 \n(na figura com difusor tipo FD)\nCaixa Terminal F650\n(na figura com difusor tipo ADLQ-A)\nCaixa Terminal F670 \n(na figura com grelha tipo AT-A)',
 '(na figura com grelha tipo AT-A)\nÍndice - Conteúdo -Tipos de produtos ........................... 2\nCaixa Terminal F631 com Filtros de Alta Eficiência para \nmontagem no teto .......................................................... 3\nCaixa Terminal F650 com Filtros de Alta Eficiência para',
 'montagem no teto .......................................................... 4\nCaixa Termina

In [6]:
def get_vectorstore(text_chunks):
    embeddings = HuggingFaceEmbeddings(model_name='multi-qa-mpnet-base-dot-v1')
    vectorstore = FAISS.from_texts(texts=text_chunks,embedding=embeddings)
    return vectorstore

In [22]:
vectorstore = get_vectorstore(text_chunks)
print(vectorstore)

In [20]:
def get_convesation_chain():
    llm = ChatGroq(model='llama-3.1-8b-instant',temperature=0.2)
    
    prompt = ChatPromptTemplate.from_messages([
        ('system',"Você é um assistente na área de HVAC (ar condicionado). Responda de forma objetiva e correta."),
        MessagesPlaceholder('history'),
        ('human','{question}')
        ])
    
    chain = prompt | llm | StrOutputParser()
    
    return chain

In [21]:
chain = get_convesation_chain()
resposta = chain.invoke({'question':'O que é um split?', 'history':[]})
print(resposta)

Um split é um tipo de sistema de ar condicionado que consiste em dois componentes principais: um condensador externo e um evaporador interno.

O condensador externo é geralmente instalado no exterior da edificação, enquanto o evaporador interno é instalado dentro da casa ou edifício, geralmente em uma janela ou em uma parede.

O condensador externo é responsável por dissipar o calor do refrigerante, enquanto o evaporador interno é responsável por resfriar o ar e transferir o calor para o condensador externo.

Os sistemas de split são conhecidos por serem silenciosos, fáceis de instalar e manter, e eficientes em termos de energia. Eles são uma opção popular para residências e pequenas empresas, pois oferecem uma solução de ar condicionado completa e eficaz sem a necessidade de uma grande unidade central.


In [24]:
retriever = vectorstore.as_retriever(search_kwargs={'k':3})

In [25]:
qa_chain = RetrievalQA.from_chain_type(
    llm=ChatGroq(model='llama-3.1-8b-instant', temperature=0.2),
    retriever=retriever,
    chain_type='stuff'
)

In [26]:
resposta_documento = qa_chain.invoke({'query': 'O que é uma caixa terminal?'})
print(resposta_documento)

{'query': 'O que é uma caixa terminal?', 'result': 'Com base no contexto fornecido, uma caixa terminal parece ser um tipo de equipamento utilizado em sistemas de ventilação ou ar condicionado. Ela é projetada para ser montada no teto e provavelmente serve como uma unidade central para controlar e filtrar o ar que circula no sistema.\n\nA presença de filtros de alta eficiência e diferentes tipos de difusores ou grelhas sugere que a caixa terminal é responsável por limpar e distribuir o ar de forma eficiente e eficaz. Além disso, a variedade de modelos (F631, F640, F660, F650 e F670) com diferentes características e configurações indica que a caixa terminal pode ser adaptada a diferentes necessidades e aplicações.\n\nNo entanto, sem mais informações, é difícil determinar com precisão a função exata de uma caixa terminal. Se você tiver mais contexto ou detalhes, posso tentar fornecer uma resposta mais precisa.'}


In [27]:
def _format_docs(docs):
    # junta os trechos retornados pelo retriever
    return "\n\n---\n\n".join(d.page_content for d in docs)

In [33]:
def build_rag_chain():
    llm = ChatGroq(model='llama-3.1-8b-instant',temperature=0.2)
    
    prompt = ChatPromptTemplate.from_messages([
        ('system',
        "Você é um assistente técnico de HVAC. Responda com base ESTRITA no contexto. "
        "Se não houver contexto suficiente, diga que não encontrou no documento."),
        ('human','Pergunta:{question}\n\n'
        'Contexto (trechos recuperados):\n{context}')
        ])
    return prompt | llm | StrOutputParser()

In [34]:
def handler_user_input(question: str):
    global rag, retriever
    if retriever:
        chain = {
            'context': retriever | RunnableLambda(_format_docs),
            'question': RunnablePassthrough()
            } | build_rag_chain
        answer = chain.invoke(question)
        return answer
    else:
        # fallback: sem contexto
        answer = rag.invoke({"question": question, "context": "(nenhum contexto: nenhum índice carregado)"})
        return answer



In [35]:
print(handler_user_input("O que é um sistema split?"))

TypeError: unsupported operand type(s) for |: 'dict' and 'function'